In [1]:
import numpy as np
import pandas as pd

import warnings
from matplotlib.colors import ListedColormap

warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report
from sklearn import metrics

In [2]:
novel_features = pd.read_csv("novel_features.csv", index_col = 0)
novel_features.head()

,Writer Name,Novel Name,Genre,Final Word Count,Daily Average,Winner,Synopses,url,Novel Date,has genre,...,num uniques,num sentences,paragraphs,fk score,has excerpt,num words excerpt,num uniques excerpt,num sentences excerpt,paragraphs excerpt,fk score excerpt
0,Nicaless,Novel: Lauren's Birthday,Genre: Young Adult,24229,807,0,\n<p></p>\n,http://nanowrimo.org/participants/nicaless/nov...,November 2015,1,...,0,0,0,0.00,0,0,0,0,0,0.00
1,Nicaless,Novel: A Mystery in the Kingdom of Aermon,Genre: Fantasy,50919,"1,697",1,\n<p>Hitoshi is appointed the youngest Judge a...,http://nanowrimo.org/participants/nicaless/nov...,November 2014,1,...,42,3,1,65.73,1,132,96,13,7,78.25
2,Rachel B. Moore,Novel: Finding Fortunato,Genre: Literary,50603,"1,686",1,\n<p>Sam and Anna Gold and their newly adoptiv...,http://nanowrimo.org/participants/rachel-b-moo...,November 2015,1,...,109,7,4,58.62,0,0,0,0,0,0.00
3,Rachel B. Moore,Novel: The Residency,Genre: Literary,50425,"1,680",1,\n<p>It's every writer's dream - an all-expens...,http://nanowrimo.org/participants/rachel-b-moo...,November 2014,1,...,51,4,3,65.73,0,0,0,0,0,0.00
4,Rachel B. Moore,Novel: The Jew From Fortunato,Genre: Literary Fiction,41447,"1,381",0,\n<p>20-something Andre Levinsky is a fish out...,http://nanowrimo.org/participants/rachel-b-moo...,November 2013,1,...,93,4,1,56.93,0,0,0,0,0,0.00


In [3]:
del novel_features['Novel Name']
del novel_features['Genre']
del novel_features['Final Word Count']
del novel_features['Daily Average']
del novel_features['Synopses']
del novel_features['url']
novel_features.head()

,Writer Name,Winner,Novel Date,has genre,standard genre,has_synopses,num words,num uniques,num sentences,paragraphs,fk score,has excerpt,num words excerpt,num uniques excerpt,num sentences excerpt,paragraphs excerpt,fk score excerpt
0,Nicaless,0,November 2015,1,0,0,0,0,0,0,0.00,0,0,0,0,0,0.00
1,Nicaless,1,November 2014,1,0,1,44,42,3,1,65.73,1,132,96,13,7,78.25
2,Rachel B. Moore,1,November 2015,1,0,1,153,109,7,4,58.62,0,0,0,0,0,0.00
3,Rachel B. Moore,1,November 2014,1,0,1,59,51,4,3,65.73,0,0,0,0,0,0.00
4,Rachel B. Moore,0,November 2013,1,0,1,124,93,4,1,56.93,0,0,0,0,0,0.00


In [ ]:
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])
df = novel_features[novel_features['has_synopses'] == 1 * novel_features['has excerpt'] == 1]
no.plot(kind='scatter', x='paragraphs', y='paragraphs excerpt', c='Winner', colormap = cmap_bold)

In [ ]:
winlose = novel_features.groupby("Winner")
df = pd.DataFrame({'loss': winlose['has excerpt'].get_group(0), 'win': winlose['has excerpt'].get_group(1)})
df.plot(kind='hist', stacked=True)

In [ ]:
excerpts = novel_features[novel_features['has excerpt'] == 0]
winners = len(excerpts[excerpts['Winner'] == 1])
nonwinners = len(excerpts[excerpts['Winner'] == 0])
print "The ratio of winners to nonwinners for those without excerpts is " + str( float(winners) / nonwinners)

In [ ]:
excerpts = novel_features[novel_features['has excerpt'] == 1]
winners = len(excerpts[excerpts['Winner'] == 1])
nonwinners = len(excerpts[excerpts['Winner'] == 0])
print "The ratio of winners to nonwinners for those with excerpts is " + str( float(winners) / nonwinners)

In [ ]:
synopsis = novel_features[novel_features['has_synopses'] == 0]
winners = len(synopsis[synopsis['Winner'] == 1])
nonwinners = len(synopsis[synopsis['Winner'] == 0])
print "The ratio of winners to nonwinners for those without synopses is " + str( float(winners) / nonwinners)

In [ ]:
synopsis = novel_features[novel_features['has_synopses'] == 1]
winners = len(synopsis[synopsis['Winner'] == 1])
nonwinners = len(synopsis[synopsis['Winner'] == 0])
print "The ratio of winners to nonwinners for those with synopses is " + str( float(winners) / nonwinners)

In [7]:
test = novel_features[novel_features['Novel Date'] == "November 2015"]
train = novel_features[novel_features['Novel Date'] != "November 2015"]
print str(sum(test['Winner'] / float(len(test)))) + " is the fraction of winners in the test data"
print str(sum(train['Winner'] / float(len(train)))) + " is the fraction of winners in the train data"

0.437125748503 is the fraction of winners in the test data
0.686806411837 is the fraction of winners in the train data


In [5]:
del train['Writer Name']
del train['Novel Date']

del test['Writer Name']
del test['Novel Date']

trainX = train
trainy = train['Winner']
del trainX['Winner']

testX = test
testy = test['Winner']
del testX['Winner']

In [8]:
trainX = train[['has_synopses', 'has excerpt']].values
trainy = train['Winner']
testX = test[['has_synopses', 'has excerpt']].values
testy = test['Winner']


In [ ]:
# create function to create matrix that is just 0 and 1 for greater than average and less than average 

In [10]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

cross_val_score(nb, trainX, trainy, cv=10).mean()

0.68680889859870908

In [ ]:
from sklearn.svm import SVC

model = SVC(C=1,kernel='rbf')
cross_val_score(model, trainX, trainy, cv = 10).mean()


In [ ]:
model_svc = SVC(C=1,kernel='rbf').fit(trainX,trainy)
print model_svc.score(testX, testy)
print classification_report(testy,model_svc.predict(testX))